In [158]:
import logging

def get_rotpos(steps, turns):
    print(f"get_rotpos: {steps/spt=}, {turns=}")
    return steps/spt + turns
    
def steps_turn_to_az(steps, turns):
    rotpos = get_rotpos(steps, turns)
    print(f"steps_turn_to_az: {rotpos=}, {rotpos-north_rotpos=}")
    return ((rotpos - north_rotpos)*degree_per_turn) % 360
    

# 0 azimuth is North
# 360 = 0
spt = 2**14
tpr = 5
degree_per_turn = 360/tpr
degree_per_step = degree_per_turn/spt
turn_per_degree = tpr/360
home_az = 90
home_steps = 10000
home_turns = 2
home_rotpos = get_rotpos(home_steps, home_turns)
north_rotpos = home_rotpos + ((0 - home_az)/degree_per_turn)
assert degree_per_step*spt == degree_per_turn
print(f"{degree_per_step=}\n{degree_per_turn=}\n{turn_per_degree=}\n{home_rotpos=}\n{north_rotpos=}")

get_rotpos: steps/spt=0.6103515625, turns=2
degree_per_step=0.00439453125
degree_per_turn=72.0
turn_per_degree=0.013888888888888888
home_rotpos=2.6103515625
north_rotpos=1.3603515625


In [159]:
print("should be 90:", steps_turn_to_az(10000, 2))

get_rotpos: steps/spt=0.6103515625, turns=2
steps_turn_to_az: rotpos=2.6103515625, rotpos-north_rotpos=1.25
should be 90: 90.0


In [160]:
steps_turn_to_az(10000, 7)

get_rotpos: steps/spt=0.6103515625, turns=7
steps_turn_to_az: rotpos=7.6103515625, rotpos-north_rotpos=6.25


90.0

In [163]:
RIGHT = 1
LEFT = 2
limits = {LEFT: -270, RIGHT:270}

def direction_finder(current_az, target_az):
    diff = abs(target_az - current_az)
    is_left = (target_az - current_az) < 0
    return is_left, diff
    #back_dir = abs(current_az + (360-target_az))
    #return forward_dir, back_dir

def new_direction_finder(current_az, target_az, limits, limitscounter):
    diff1 = (target_az - current_az)%360
    if diff1 == 0:
        direction = RIGHT
        diff = 0
    if diff1 == 180:
        print("180 degrees, going towards the shortest cable")
        direction = RIGHT if limitscounter[RIGHT] < abs(limitscounter[LEFT]) else LEFT
    if current_az + 180 > target_az:
        # Rotate current directly towards target.
        direction = RIGHT
        diff = diff1
    else:
        # Rotate the other direction towards target.
        direction = LEFT
        diff = current_az + 360 - target_az

    print(f"Orig result: {'LEFT' if direction==LEFT else 'RIGHT'}, {diff=}")
    if direction == LEFT:
        total_left = limitscounter[LEFT] - diff
        print(f"{total_left=}, {limitscounter[RIGHT]=}")
        if total_left+limitscounter[RIGHT] < limits[LEFT]:
            print("cable length violation")
            direction = RIGHT
            diff = 360 - diff
    else:
        total_right = limitscounter[RIGHT] + diff
        print(f"{limitscounter[LEFT]=}, {total_right=}")
        if limitscounter[LEFT]+total_right > limits[RIGHT]:
            print("cable length violation")
            direction = LEFT
            diff = 360 - diff
        
    return direction, diff

import math

def shortest_path(currentaz, targetaz):
    diff_rad = math.radians(currentaz) - math.radians(targetaz)
    diff_rad2 = math.radians(currentaz) - math.radians(targetaz)
    return diff_rad, diff_rad2, math.degrees(abs(diff_rad))
    
print(shortest_path(90,350))
print(math.radians(90), math.radians(350))

(-4.537856055185257, -4.537856055185257, 260.0)
1.5707963267948966 6.1086523819801535


In [164]:

t1=(90,100)
r1=(RIGHT, 10)
t2=(90,350)
r2=(LEFT, 100)
t3=(350,10)
r3=(RIGHT, 20)
t4=(359,0)
r4=(RIGHT, 1)
t5=(0,180)
r5=(RIGHT, 180)
t6=(10,359)
r6=(LEFT, 11)


tests = [t1, t2, t3, t4, t5, t6]
results = [r1, r2, r3, r4, r5, r6]

print(limits)
print(limitscounter)
for idx, test in enumerate(tests):
    limitscounter = {LEFT: -260, RIGHT: 0}
    direction, diff = new_direction_finder(*test, limits, limitscounter)
    
    print(f"TEST: {test=}, {'LEFT' if direction == LEFT else 'RIGHT'}, {diff=}, {direction == results[idx][0] and diff == results[idx][1]}")
    print(f"Expected: {'LEFT' if results[idx][0] == LEFT else 'RIGHT'}, {results[idx][1]}\n")
    



{2: -270, 1: 270}
{2: -260, 1: 0}
Orig result: RIGHT, diff=10
limitscounter[LEFT]=-260, total_right=10
TEST: test=(90, 100), RIGHT, diff=10, True
Expected: RIGHT, 10

Orig result: LEFT, diff=100
total_left=-360, limitscounter[RIGHT]=0
cable length violation
TEST: test=(90, 350), RIGHT, diff=260, False
Expected: LEFT, 100

Orig result: RIGHT, diff=20
limitscounter[LEFT]=-260, total_right=20
TEST: test=(350, 10), RIGHT, diff=20, True
Expected: RIGHT, 20

Orig result: RIGHT, diff=1
limitscounter[LEFT]=-260, total_right=1
TEST: test=(359, 0), RIGHT, diff=1, True
Expected: RIGHT, 1

180 degrees, going towards the shortest cable
Orig result: LEFT, diff=180
total_left=-440, limitscounter[RIGHT]=0
cable length violation
TEST: test=(0, 180), RIGHT, diff=180, True
Expected: RIGHT, 180

Orig result: LEFT, diff=11
total_left=-271, limitscounter[RIGHT]=0
cable length violation
TEST: test=(10, 359), RIGHT, diff=349, False
Expected: LEFT, 11



In [62]:
isinstance('N', str)

True

In [ ]:
cables = {"LEFT": 0, "LEFT_MAX": -270, "RIGHT": 0, "RIGHT_MAX": 270}




In [120]:
700%360

340

In [49]:
from dataclasses import dataclass

@dataclass(init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False)
class DomePos:
    # don't change externally!
    steps: int
    turns: int
    az: float
    rotpos: float = -1

    def __init__(self, steps, turns, az):
        self.steps = steps
        self.turns = turns
        self.az = az

    @property
    def alt(self) -> int:
        return 0


In [50]:
a = DomePos(steps=0, turns=1, az=90)

In [51]:
a

DomePos(steps=0, turns=1, az=90, rotpos=-1)

# Figuring out atan2 for angle calculations

In [1]:
from math import radians, degrees, atan2

In [2]:
def calc_angle(angle1, angle2):
    return degrees(atan2(radians(angle1), radians(angle2)))

calc_angle(90,180)


26.56505117707799

In [14]:
import os
print(os.getcwd())
os.chdir('..')


/Users/mike/dev/astropolis/koepel/dome-alpaca/notebooks


In [121]:
import math
def smallest_diff(angle1, angle2):
    """ smallest diff between 2 angles, including sign (minus is LEFT rotate) """
    diff1 = (angle1 - angle2)%360
    diff2 = (angle2 - angle1)%360
    return min(diff1, diff2)

def smallest_diff_signed(angle1, angle2):
    """ smallest diff between 2 angles, including sign (minus is LEFT rotate) """
    anglediff1 = (angle1 - angle2)
    anglediff2 = (angle2 - angle1)
    diff1 = anglediff1%360
    diff1sgn = math.copysign(1, anglediff1)
    diff2 = anglediff2%360
    diff2sgn = math.copysign(1, anglediff2)
    return diff1sgn*diff1, diff2sgn*diff2

def smallest_diff_signed2(angle1, angle2):
    """ smallest diff between 2 angles, including sign (minus is LEFT rotate) """
    anglediff1 = (angle1 - angle2)
    anglediff2 = (angle2 - angle1)
    print(f"1-2: {anglediff1},{anglediff2} ")
    diff1 = (angle1 - angle2)%360
    print(f"{diff1=}, {angle2-angle1}")
    diff2 = (angle2 - angle1)%360
    print(f"{diff2=}")
    if diff1 < diff2:
        return math.copysign(1, anglediff1)*diff1
    else:
        return math.copysign(1, anglediff2)*diff2

def oldway(start, end):
    print(f"FROM {start} TO {end}")
    start_end = (start - end)
    end_start = (end - start)
    bigstart_end = start+360-end
    end_start360 = (end - start)%360
    print(f"1-2={start_end}, 2-1={end_start}, bigstart={bigstart_end}, 2-1%360={end_start360}")
    # if abs(end_start) < end_start360:
    #     return end_start
    # else:
    #     return end_start360
                
    if end < start or start + 180 > end:
        # Rotate current directly towards target.
        direction = 1
        diff = end_start360
        print(f"CASE1: {direction}, {diff}")
    else:
        # Rotate the other direction towards target.
        direction = -1
        diff = start + 360 - end
        print(f"CASE2: {direction}, {diff}")
    return math.copysign(1, direction) * diff

def newway1(first, second):
    # https://stackoverflow.com/questions/9505862/shortest-distance-between-two-degree-marks-on-a-circle
    print(f"FROM {first} TO {second}")
    firstbiggest = -1 if first > second else 1
    print(f"{firstbiggest=}")
    raw_diff = first - second if first > second else second - first
    print(f"{raw_diff=}")
    mod_diff = math.fmod(raw_diff, 360.0);
    print(f"{mod_diff=}")
    dist = -(360 - mod_diff) if mod_diff > 180.0 else mod_diff*math.copysign(1,firstbiggest)
    return dist

def newway(origin, target):
    MAX_VALUE=360.0

    signedDiff = 0.0;
    raw_diff = origin - target if origin > target else target - origin
    mod_diff = math.fmod(raw_diff, MAX_VALUE);

    if(mod_diff > (MAX_VALUE/2) ):
        # There is a shorter path in opposite direction
        signedDiff = (MAX_VALUE - mod_diff)
        if(target>origin): signedDiff = signedDiff * -1;
    else:
        signedDiff = mod_diff;
        if(origin>target): signedDiff = signedDiff * -1;

    return signedDiff;


In [122]:
# print(f"result={smallest_diff_signed2(90,180)}, should be {90}\n")
# print(f"result={smallest_diff_signed2(180,90)}, should be {-90}\n")
# print(f"result={smallest_diff_signed2(350,14)}, should be {24}\n")
# print(f"result={smallest_diff_signed2(14, 350)}, should be {-24}\n")

# print(f"result={oldway(90,180)}, should be {90}\n")
# print(f"result={oldway(180,90)}, should be {-90}\n")
# print(f"result={oldway(350,14)}, should be {24}\n")
# print(f"result={oldway(14, 350)}, should be {-24}\n")

print(f"result={newway(90,180)}, should be {90}\n")
print(f"result={newway(180,90)}, should be {-90}\n")
print(f"result={newway(350,14)}, should be {24}\n")
print(f"result={newway(14, 350)}, should be {-24}\n")



result=90.0, should be 90

result=-90.0, should be -90

result=24.0, should be 24

result=-24.0, should be -24



In [39]:
math.fmod(-90, 180)

-90.0

In [107]:
math.sin(math.radians(90))

1.0